In [1]:
import warnings
warnings.filterwarnings("ignore")

from fbprophet import Prophet
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

from scipy import stats
import statsmodels.api as sm
from itertools import product
from datetime import timedelta

import pandas as pd
pd.options.display.max_columns = 999
import numpy as np

from sklearn.metrics import mean_squared_error
#RMSE = mean_squared_error(y, y_pred)**0.5

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import graph_objs as go
%matplotlib inline
import matplotlib.pyplot as plt

In [37]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## data preparation

In [130]:
df = pd.read_csv('data/player_price.csv')
df['ds'] = pd.to_datetime(df['timestamp'], unit='ms')
df=df[df.ds>='2017-06-01']
df=df.loc[df["player_id"] == 1]
df.drop(["timestamp", "ps_price", "player_name", "player_id"], axis=1, inplace=True)
df.rename(columns={"xbox_price": "y"}, inplace=True)
df=df.replace(0,1)
print(df.shape)
df.head(10)

(22, 2)


,y,ds
251,3227500,2017-06-01
252,3115720,2017-06-02
253,3263857,2017-06-03
254,3351619,2017-06-04
255,3132091,2017-06-05
256,3535840,2017-06-06
257,3508952,2017-06-07
258,3515515,2017-06-08
259,3289895,2017-06-09
260,3258917,2017-06-10


In [122]:
#id-шники для предикшина
sample_subm = pd.read_csv('data/kaggle_sample_submission.csv')
sample_subm["player_id"] = sample_subm["id"].apply(lambda x: int(x.split("_")[0]))
player_id=sample_subm.groupby(['player_id'])['price'].count()
player_id.index

Int64Index([    1,     2,     3,     4,     5,     6,     7,     8,     9,
               10,
            ...
            18336, 18337, 18338, 18339, 18340, 18341, 18342, 18344, 18345,
            18370],
           dtype='int64', name='player_id', length=1253)

## prediction

In [131]:
predictions = 7
#train_df=df
train_df = df[:-predictions]

In [132]:
%%time
m = Prophet()
m.fit(train_df)
future = m.make_future_dataframe(periods=predictions)
forecast = m.predict(future)
future.tail(7)

Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
Wall time: 2.45 s


In [133]:
cmp_df = forecast.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']].join(df.set_index('ds'))

cmp_df['e'] = cmp_df['y'] - cmp_df['yhat']
cmp_df['p'] = 100*cmp_df['e']/cmp_df['y']
print('MAPE', np.mean(abs(cmp_df[-predictions:]['p'])))
print('MAE', np.mean(abs(cmp_df[-predictions:]['e'])))
print(mean_squared_error(df.y[-7:], cmp_df.yhat[-7:])**0.5)

MAPE 2.6409404643443053
MAE 85188.06189147184
100058.954928


---

### validation

In [141]:
predictions = 7
train_df = df[:-predictions]

In [142]:
%%time
m = Prophet()
m.fit(train_df)
future = m.make_future_dataframe(periods=predictions)
forecast = m.predict(future)
future.tail(7)

Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
Wall time: 1.93 s


In [143]:
cmp_df = forecast.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']].join(df.set_index('ds'))
cmp_df.tail(7)

,yhat,yhat_lower,yhat_upper,y
ds,,,,
2017-06-16,3.383420e+06,3.232611e+06,3.533953e+06,3212762
2017-06-17,3.420175e+06,3.272220e+06,3.576612e+06,3312852
2017-06-18,3.427870e+06,3.283284e+06,3.573526e+06,3227893
2017-06-19,3.433423e+06,3.282959e+06,3.584672e+06,3142655
2017-06-20,3.458722e+06,3.306969e+06,3.606256e+06,3135387
2017-06-21,3.477677e+06,3.326052e+06,3.634103e+06,3248943
2017-06-22,3.485954e+06,3.331920e+06,3.645646e+06,3137143


In [121]:
qwe=cmp_df.yhat[-7:]    
#print(qwe)

pp=pd.DataFrame(columns=['y','ds'])
pp['ds']=qwe.index.astype(str)
pp.y=qwe.values
pp.y=pp.y.astype(int)
pp

,y,ds
0,3097255,2017-06-16
1,3136083,2017-06-17
2,3129383,2017-06-18
3,3087740,2017-06-19
4,3240035,2017-06-20
5,3206386,2017-06-21
6,3133730,2017-06-22


In [122]:
df_new=train_df
df_new.tail()

,y,ds
261,3196091,2017-06-11
262,3369791,2017-06-12
263,3311636,2017-06-13
264,3308667,2017-06-14
265,3269308,2017-06-15


In [124]:
df_new=df_new.append(pp,ignore_index=True)
df_new.tail(10)

,y,ds
12,3311636,2017-06-13 00:00:00
13,3308667,2017-06-14 00:00:00
14,3269308,2017-06-15 00:00:00
15,3097255,2017-06-16
16,3136083,2017-06-17
17,3129383,2017-06-18
18,3087740,2017-06-19
19,3240035,2017-06-20
20,3206386,2017-06-21
21,3133730,2017-06-22


In [125]:
%%time
m = Prophet()
m.fit(df_new)
future = m.make_future_dataframe(periods=7)
forecast = m.predict(future)
future.tail(7)

Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
Wall time: 1.99 s


In [126]:
cmp_df_new = forecast.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']].join(df.set_index('ds'))
cmp_df_new.tail(7)

,yhat,yhat_lower,yhat_upper,y
ds,,,,
2017-06-23,2.948646e+06,2.871044e+06,3.026023e+06,NaN
2017-06-24,2.987141e+06,2.909318e+06,3.069214e+06,NaN
2017-06-25,2.981143e+06,2.899615e+06,3.061322e+06,NaN
2017-06-26,2.939954e+06,2.859732e+06,3.018517e+06,NaN
2017-06-27,3.093834e+06,3.008773e+06,3.176259e+06,NaN
2017-06-28,3.061287e+06,2.975877e+06,3.146325e+06,NaN
2017-06-29,2.986164e+06,2.905922e+06,3.074350e+06,NaN


---

In [7]:
# 2017-02-01 prediction_fbpophet9
# 2017-06-01 prediction_fbpophet10
# 2017-06-15 prediction_fbpophet11
# 2017-06-18 prediction_fbpophet12
# 2017-06-15 prediction_fbpophet13 0=median
# 2017-06-15 prediction_fbpophet14 m = Prophet(changepoint_prior_scale=0.3, interval_width=0.3)

In [111]:
df_all = pd.read_csv('data/player_price.csv')
df_all['ds'] = pd.to_datetime(df_all['timestamp'], unit='ms')
df_all=df_all[df_all.ds>='2017-06-15']
df_all.drop(["timestamp", "ps_price", "player_name"], axis=1, inplace=True)
df_all.rename(columns={"xbox_price": "y"}, inplace=True)
print(df_all.shape)
df_all.head()

(36786, 3)


,y,player_id,ds
265,3269308,1,2017-06-15
266,3212762,1,2017-06-16
267,3312852,1,2017-06-17
268,3227893,1,2017-06-18
269,3142655,1,2017-06-19


In [112]:
#id-шники для предикшина
sample_subm = pd.read_csv('data/kaggle_sample_submission.csv')
sample_subm["player_id"] = sample_subm["id"].apply(lambda x: int(x.split("_")[0]))
player_id=sample_subm.groupby(['player_id'])['price'].count()
#player_id.index

In [104]:
player_id.index[349]

3353

In [114]:
%%time
k=1
for i in player_id.index[350:len(player_id.index)]:#player_id.index
    print('==================================')
    print('id={}'.format(i))
       
    df=df_all.loc[df_all['player_id'] == i]
    mediana=df.y[df.player_id==i].median()
    df.drop(['player_id'], axis=1, inplace=True)
    #df=df.replace(0,1)    
    df=df.replace(0,int(mediana))
    
    train_df=df
    
    #m = Prophet()
    m=Prophet()
    m.fit(train_df)
    
    future = m.make_future_dataframe(periods=7)
    forecast = m.predict(future)
    
    cmp_df = forecast.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']].join(df.set_index('ds'))
    
    qwe=cmp_df.yhat[-7:]    
    #print(qwe)
    
    pp=pd.DataFrame(columns=['id','price'])
    pp['id']=str(i)+'_'+qwe.index.astype(str)
    pp.price=qwe.values
    pp.price=pp.price.astype(int)
    print(pp)
    
    if k==0:
        pp.to_csv('csv/prediction_fbpophet14.csv',sep=',',header=True,index=False)
        k=1
    else:
        with open('csv/prediction_fbpophet14.csv', 'a') as f:
            pp.to_csv(f, header=False,index=False,sep=',')

id=3365
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
                id  price
0  3365_2017-06-23   3295
1  3365_2017-06-24   2993
2  3365_2017-06-25   2692
3  3365_2017-06-26   2390
4  3365_2017-06-27   2089
5  3365_2017-06-28   1787
6  3365_2017-06-29   1485
id=3376
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
                id  price
0  3376_2017-06-23   3572
1  3376_2017-06-24   3280
2  3376_2017-06-25   2988
3  3376_2017-06-26   2696
4  3376_2017-06-27   2405
5  3376_2017-06-28   2113
6  3376_2017-06-29   1821
id=3389
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
                id  price


                id  price
0  4081_2017-06-23  11877
1  4081_2017-06-24  12053
2  4081_2017-06-25  12228
3  4081_2017-06-26  12404
4  4081_2017-06-27  12579
5  4081_2017-06-28  12755
6  4081_2017-06-29  12931
id=4095
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
                id  price
0  4095_2017-06-23   5010
1  4095_2017-06-24   5068
2  4095_2017-06-25   5126
3  4095_2017-06-26   5183
4  4095_2017-06-27   5241
5  4095_2017-06-28   5299
6  4095_2017-06-29   5357
id=4139
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
                id  price
0  4139_2017-06-23   4536
1  4139_2017-06-24   4660
2  4139_2017-06-25   4784
3  4139_2017-06-26   4907
4  4139_2017-06-27   5031
5  4139_2017-06-28   5155
6  4139_2017-06-29   5278
id=4

                id  price
0  4956_2017-06-23   1137
1  4956_2017-06-24   1112
2  4956_2017-06-25   1088
3  4956_2017-06-26   1063
4  4956_2017-06-27   1039
5  4956_2017-06-28   1014
6  4956_2017-06-29    989
id=5152
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
                id  price
0  5152_2017-06-23   5500
1  5152_2017-06-24   5781
2  5152_2017-06-25   6061
3  5152_2017-06-26   6342
4  5152_2017-06-27   6622
5  5152_2017-06-28   6903
6  5152_2017-06-29   7184
id=5412
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
                id  price
0  5412_2017-06-23   5723
1  5412_2017-06-24   5901
2  5412_2017-06-25   6079
3  5412_2017-06-26   6257
4  5412_2017-06-27   6436
5  5412_2017-06-28   6614
6  5412_2017-06-29   6792
id=5

                 id  price
0  11793_2017-06-23   3718
1  11793_2017-06-24   4268
2  11793_2017-06-25   4817
3  11793_2017-06-26   5366
4  11793_2017-06-27   5916
5  11793_2017-06-28   6465
6  11793_2017-06-29   7014
id=14872
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
                 id  price
0  14872_2017-06-23    877
1  14872_2017-06-24    891
2  14872_2017-06-25    904
3  14872_2017-06-26    918
4  14872_2017-06-27    931
5  14872_2017-06-28    945
6  14872_2017-06-29    958
id=14969
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
                 id  price
0  14969_2017-06-23   1092
1  14969_2017-06-24   1081
2  14969_2017-06-25   1070
3  14969_2017-06-26   1060
4  14969_2017-06-27   1049
5  14969_2017-06-28   1038
6  14

                 id  price
0  15090_2017-06-23  18142
1  15090_2017-06-24  19475
2  15090_2017-06-25  20808
3  15090_2017-06-26  22141
4  15090_2017-06-27  23474
5  15090_2017-06-28  24807
6  15090_2017-06-29  26140
id=15091
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
                 id   price
0  15091_2017-06-23  875058
1  15091_2017-06-24  891962
2  15091_2017-06-25  908866
3  15091_2017-06-26  925770
4  15091_2017-06-27  942674
5  15091_2017-06-28  959578
6  15091_2017-06-29  976482
id=15092
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
                 id   price
0  15092_2017-06-23  114021
1  15092_2017-06-24  115043
2  15092_2017-06-25  116065
3  15092_2017-06-26  117087
4  15092_2017-06-27  118109
5  15092_2017-06-2

...

                 id  price
0  18191_2017-06-23  15290
1  18191_2017-06-24  15352
2  18191_2017-06-25  15414
3  18191_2017-06-26  15476
4  18191_2017-06-27  15538
5  18191_2017-06-28  15600
6  18191_2017-06-29  15662
id=18192
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
                 id  price
0  18192_2017-06-23  12490
1  18192_2017-06-24  12569
2  18192_2017-06-25  12648
3  18192_2017-06-26  12727
4  18192_2017-06-27  12806
5  18192_2017-06-28  12885
6  18192_2017-06-29  12964
id=18194
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
                 id  price
0  18194_2017-06-23  65862
1  18194_2017-06-24  66077
2  18194_2017-06-25  66292
3  18194_2017-06-26  66507
4  18194_2017-06-27  66722
5  18194_2017-06-28  66937
6  18

                 id   price
0  18226_2017-06-23  103364
1  18226_2017-06-24   97862
2  18226_2017-06-25   92360
3  18226_2017-06-26   86858
4  18226_2017-06-27   81356
5  18226_2017-06-28   75854
6  18226_2017-06-29   70353
id=18262
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
                 id  price
0  18262_2017-06-23  20747
1  18262_2017-06-24  21001
2  18262_2017-06-25  21255
3  18262_2017-06-26  21509
4  18262_2017-06-27  21763
5  18262_2017-06-28  22017
6  18262_2017-06-29  22271
id=18264
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
                 id   price
0  18264_2017-06-23  219621
1  18264_2017-06-24  224727
2  18264_2017-06-25  229832
3  18264_2017-06-26  234938
4  18264_2017-06-27  240044
5  18264_2017-06-2

                 id   price
0  18291_2017-06-23  373986
1  18291_2017-06-24  377129
2  18291_2017-06-25  380272
3  18291_2017-06-26  383415
4  18291_2017-06-27  386558
5  18291_2017-06-28  389701
6  18291_2017-06-29  392844
id=18292
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
                 id  price
0  18292_2017-06-23  19222
1  18292_2017-06-24  20179
2  18292_2017-06-25  21136
3  18292_2017-06-26  22093
4  18292_2017-06-27  23049
5  18292_2017-06-28  24006
6  18292_2017-06-29  24963
id=18293
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
                 id  price
0  18293_2017-06-23  22826
1  18293_2017-06-24  26137
2  18293_2017-06-25  29448
3  18293_2017-06-26  32759
4  18293_2017-06-27  36070
5  18293_2017-06-28  393

                 id  price
0  18314_2017-06-23  17675
1  18314_2017-06-24  18131
2  18314_2017-06-25  18587
3  18314_2017-06-26  19043
4  18314_2017-06-27  19499
5  18314_2017-06-28  19955
6  18314_2017-06-29  20411
id=18315
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
                 id  price
0  18315_2017-06-23  14917
1  18315_2017-06-24  14133
2  18315_2017-06-25  13349
3  18315_2017-06-26  12565
4  18315_2017-06-27  11781
5  18315_2017-06-28  10997
6  18315_2017-06-29  10214
id=18316
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
                 id  price
0  18316_2017-06-23  20493
1  18316_2017-06-24  18817
2  18316_2017-06-25  17141
3  18316_2017-06-26  15465
4  18316_2017-06-27  13789
5  18316_2017-06-28  12113
6  18

                 id  price
0  18338_2017-06-23  12672
1  18338_2017-06-24  12645
2  18338_2017-06-25  12618
3  18338_2017-06-26  12591
4  18338_2017-06-27  12564
5  18338_2017-06-28  12537
6  18338_2017-06-29  12510
id=18339
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
                 id  price
0  18339_2017-06-23  15336
1  18339_2017-06-24  15760
2  18339_2017-06-25  16184
3  18339_2017-06-26  16608
4  18339_2017-06-27  17032
5  18339_2017-06-28  17456
6  18339_2017-06-29  17880
id=18340
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
                 id  price
0  18340_2017-06-23  14321
1  18340_2017-06-24  14547
2  18340_2017-06-25  14773
3  18340_2017-06-26  14999
4  18340_2017-06-27  15225
5  18340_2017-06-28  15451
6  18

In [99]:
id=3353
print(int(df_all.y[df_all.player_id==id].median()))
df_all[df_all.player_id==id]

0


,y,player_id,ds
668551,0,3353,2017-06-15
668552,0,3353,2017-06-16
668553,0,3353,2017-06-17
668554,0,3353,2017-06-18
668555,0,3353,2017-06-19
668556,0,3353,2017-06-20
668557,0,3353,2017-06-21
668558,0,3353,2017-06-22


In [56]:
df = pd.read_csv('data/player_price.csv')

In [100]:
df['ds'] = pd.to_datetime(df['timestamp'], unit='ms')
df.rename(columns={"xbox_price": "y"}, inplace=True)
print(df.y[df.player_id==id].median())
df[df.player_id==id].tail(10)

KeyError: 'timestamp'

In [101]:
qwe = pd.read_csv('data/player_price.csv')

In [102]:
qwe.xbox_price[qwe.player_id==id].median()

4950.0